In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.tsv', 'sampleSubmission.csv', 'train.tsv']


In [2]:
train_df=pd.read_csv('../input/train.tsv',sep='\t')
test_df=pd.read_csv('../input/test.tsv',sep='\t')

In [3]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
from nltk.tokenize.treebank import TreebankWordTokenizer

In [5]:
tbwt=TreebankWordTokenizer()
def tokenizer(row):
    ltokens=tbwt.tokenize(row['Phrase'])
    #print(row['Phrase'])
    tok_sen=' '.join(ltokens)
    #print(ltokens)
    return tok_sen

In [6]:
train_df['token']=train_df.apply(tokenizer,axis=1)
test_df['token']=test_df.apply(tokenizer,axis=1)


In [7]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment,token
0,1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...
1,2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...
2,3,1,A series,2,A series
3,4,1,A,2,A
4,5,1,series,2,series


In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(row):
    #print(row['token'])
    split=row['token'].split(' ')
    lem_sen=' '.join([lemmatizer.lemmatize(w) for w in split])
    #print(lem_sen)
    return lem_sen

In [9]:
train_df['lemma']=train_df.apply(lemmatize,axis=1)
test_df['lemma']=test_df.apply(lemmatize,axis=1)

In [10]:
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment,token,lemma
0,1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...,A series of escapade demonstrating the adage t...
1,2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...,A series of escapade demonstrating the adage t...
2,3,1,A series,2,A series,A series
3,4,1,A,2,A,A
4,5,1,series,2,series,series


In [11]:
from keras.preprocessing.text import Tokenizer
token=Tokenizer()
token.fit_on_texts(train_df['lemma'])

Using TensorFlow backend.


In [12]:
X_train=token.texts_to_sequences(train_df['lemma'])
X_train

[[2,
  333,
  3,
  6551,
  5738,
  1,
  6257,
  10,
  55,
  9,
  47,
  14,
  1,
  2884,
  9,
  185,
  47,
  14,
  1,
  10171,
  66,
  3,
  80,
  663,
  9454,
  17,
  591,
  3,
  80,
  664,
  5,
  58,
  3,
  2,
  40],
 [2, 333, 3, 6551, 5738, 1, 6257, 10, 55, 9, 47, 14, 1, 2884],
 [2, 333],
 [2],
 [333],
 [3, 6551, 5738, 1, 6257, 10, 55, 9, 47, 14, 1, 2884],
 [3],
 [6551, 5738, 1, 6257, 10, 55, 9, 47, 14, 1, 2884],
 [6551],
 [5738, 1, 6257, 10, 55, 9, 47, 14, 1, 2884],
 [5738, 1, 6257],
 [5738],
 [1, 6257],
 [1],
 [6257],
 [10, 55, 9, 47, 14, 1, 2884],
 [10],
 [55, 9, 47, 14, 1, 2884],
 [55],
 [9, 47, 14, 1, 2884],
 [9],
 [47, 14, 1, 2884],
 [47],
 [14, 1, 2884],
 [14],
 [1, 2884],
 [2884],
 [9,
  185,
  47,
  14,
  1,
  10171,
  66,
  3,
  80,
  663,
  9454,
  17,
  591,
  3,
  80,
  664,
  5,
  58,
  3,
  2,
  40],
 [9,
  185,
  47,
  14,
  1,
  10171,
  66,
  3,
  80,
  663,
  9454,
  17,
  591,
  3,
  80,
  664,
  5,
  58,
  3,
  2,
  40],
 [9, 185],
 [185],
 [47,
  14,
  1,
  10171

In [13]:
def mlen(row):
    s=row['lemma'].split(' ')
    return len(s)

train_df['len']=train_df.apply(mlen,axis=1)
train_df.head()

,PhraseId,SentenceId,Phrase,Sentiment,token,lemma,len
0,1,1,A series of escapades demonstrating the adage ...,1,A series of escapades demonstrating the adage ...,A series of escapade demonstrating the adage t...,37
1,2,1,A series of escapades demonstrating the adage ...,2,A series of escapades demonstrating the adage ...,A series of escapade demonstrating the adage t...,14
2,3,1,A series,2,A series,A series,2
3,4,1,A,2,A,A,1
4,5,1,series,2,series,series,1


In [14]:
max(train_df['len'])

53

In [15]:
test_df['len']=test_df.apply(mlen,axis=1)
max(test_df['len'])

56

In [16]:
from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_train,maxlen=56,padding='post')

In [17]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
Y_train=train_df.iloc[:,3]
Y_train=Y_train.as_matrix()
Y_train=Y_train.reshape(-1,1)
Y_tr=ohe.fit_transform(Y_train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [18]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM

In [19]:
len(token.word_index)

14219

In [21]:
model=Sequential()
model.add(Embedding(14219,300))
model.add(LSTM(150))
model.add(Dense(5,activation='softmax'))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         4265700   
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               270600    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 755       
Total params: 4,537,055
Trainable params: 4,537,055
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit(X_train,Y_tr,batch_size=64,epochs=15)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
156060/156060 [==============================] - 149s 955us/step - loss: 1.1653 - acc: 0.5412
Epoch 2/100
156060/156060 [==============================] - 147s 945us/step - loss: 0.8194 - acc: 0.6647
Epoch 3/100
156060/156060 [==============================] - 147s 941us/step - loss: 0.7170 - acc: 0.7029
Epoch 4/100
156060/156060 [==============================] - 147s 940us/step - loss: 0.6602 - acc: 0.7237
Epoch 5/100
156060/156060 [==============================] - 146s 936us/step - loss: 0.6166 - acc: 0.7400
Epoch 6/100
156060/156060 [==============================] - 147s 939us/step - loss: 0.5782 - acc: 0.7543
Epoch 7/100
156060/156060 [==============================] - 147s 940us/step - loss: 0.5438 - acc: 0.7666
Epoch 8/100
156060/156060 [==============================] - 146s 937us/step - loss: 0.5104 - acc: 0.7785
Epoch 9/100
156060/156060 [==============================] - 146s 935us/step - loss: 0.4783 - acc: 0.789

KeyboardInterrupt: 

In [30]:
X_test=token.texts_to_sequences(test_df['lemma'])
X_test=pad_sequences(X_test,maxlen=56,padding='post')

In [31]:
X_test.shape

(66292, 56)

In [32]:
s=model.predict(X_test)
s=np.argmax(s,axis=1)
print(s)
s=pd.DataFrame(s)
s['PhraseId']=test_df['PhraseId']
s.columns=['Sentiment','PhraseId']
s=s[['PhraseId','Sentiment']]
s.to_csv('submissions.csv',index=False)

[3 3 2 ... 1 1 2]


In [33]:
s.head()

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,2
4,156065,2
